In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

In [3]:
%pip install gdown

In [4]:
RANDOM_STATE = 42

In [5]:
CARS_FILE_ID = '1liFEe1-yFISPSpRSvbv1wIH_avYNGmBI'

random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

!gdown --id {CARS_FILE_ID}

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1liFEe1-yFISPSpRSvbv1wIH_avYNGmBI
From (redirected): https://drive.google.com/uc?id=1liFEe1-yFISPSpRSvbv1wIH_avYNGmBI&confirm=t&uuid=27256348-6c25-4b37-aa9b-68da2bc445db
To: /content/dataset.csv
100% 1.01G/1.01G [00:17<00:00, 57.7MB/s]


In [6]:
df = pd.read_csv('dataset.csv')
print(df.shape)

(604047, 24)


In [7]:
df_with_na_column = df[df['engine_displacement'].isna()]
df_with_na_column

,production_year,mileage,condition,owners_number,pts_original,horse_power,accidents_resolution,region,seller_type,brand,...,engine_displacement,engine_power,fuel_rate,steering_wheel,price,price_segment,tags,auto_class,equipment,complectation_available_options
210905,2003,175000,CONDITION_OK,0,True,NaN,NaN,Чебаркуль,PRIVATE,Hyundai,...,NaN,NaN,NaN,LEFT,380000,MEDIUM,available_for_checkup;pts_original;real_photo;...,NaN,seats-5,NaN


In [8]:
df.drop(210905, inplace=True)

In [9]:
df['fuel_rate'].fillna(df['fuel_rate'].median(), inplace=True)

<ipython-input-9-b50b155cf67a>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['fuel_rate'].fillna(df['fuel_rate'].median(), inplace=True)


In [10]:
df['pts_original'].fillna(True, inplace=True)
df['accidents_resolution'].fillna('OK', inplace=True)
df['auto_class'].fillna('NOT SPECIFIED', inplace=True)

<ipython-input-10-f268849e3c66>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['pts_original'].fillna(True, inplace=True)
<ipython-input-10-f268849e3c66>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pts_original'].fillna(True, inplace=True)
<ipython-input-10-f268849e3c66>:2: FutureWarning: A value 

In [11]:
df.drop('horse_power', axis=1, inplace=True)

In [12]:
def get_unique_values(series, sep=';'):
    uniq = set()
    for cell in series.dropna():
        for piece in cell.split(sep):
            s = piece.strip()
            if s:
                uniq.add(s)
    return np.array(list(uniq))

all_tags = get_unique_values(df['tags'])
all_options = get_unique_values(df['complectation_available_options'])
all_equipments = get_unique_values(df['equipment'])

In [13]:
def create_binary_features(df, column, unique_values, sep=';'):
    return (
        df[column]
        .str.get_dummies(sep=sep)
        .reindex(columns=unique_values, fill_value=0)
        .astype('int8')
    )

tags_dummies = create_binary_features(df, 'tags', all_tags)
options_dummies = create_binary_features(df, 'complectation_available_options', all_options)
equipment_dummies = create_binary_features(df, 'equipment', all_equipments)

In [14]:
tags_dummies = tags_dummies.astype('Sparse[int]')
options_dummies = options_dummies.astype('Sparse[int]')
equipment_dummies = equipment_dummies.astype('Sparse[int]')

In [15]:
full_df = pd.concat([
    df.drop(columns=['tags', 'complectation_available_options', 'equipment']),
    tags_dummies,
    options_dummies.drop(columns=['condition']),
    equipment_dummies.drop(columns=['condition'])
], axis=1)

In [16]:
duplicated_cols = full_df.columns[full_df.columns.duplicated()].unique()
for col in duplicated_cols:
    full_df[col] = full_df[col].astype("int8")

In [17]:
import numpy as np

names = full_df.columns[full_df.columns.duplicated()].unique()
new_cols = {}

for name in names:
    cols_i = [col for col in full_df.columns if col == name]
    sub_df = full_df.loc[:, cols_i].astype('int8')   # DataFrame shape (n_rows, k)

    # берём максимум по строкам → Series shape (n_rows,)
    max_series = sub_df.max(axis=1).astype('int8')

    new_cols[name] = max_series

    # удаляем *все* старые колонки с этим именем
    full_df.drop(columns=cols_i, inplace=True)

# теперь присваиваем новые колонки–Series (они одномерны!)
for name, s in new_cols.items():
    full_df[name] = s


<ipython-input-17-65bc9051fcec>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[name] = s
<ipython-input-17-65bc9051fcec>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[name] = s
<ipython-input-17-65bc9051fcec>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[name] = s
<

In [18]:
full_df.sample(10)

,production_year,mileage,condition,owners_number,accidents_resolution,region,seller_type,brand,model,body_type,...,steel-wheels,ptf,central-airbag,esp,body-kit,mirrors-heat,immo,lane-keeping-assist,drl,projection-display
436947,2024,0,CONDITION_OK,0,OK,Санкт-Петербург,COMMERCIAL,Skoda,Karoq,ALLROAD_5_DOORS,...,0,0,0,1,0,1,1,0,1,0
329729,2020,40460,CONDITION_OK,1,ERROR,Москва,PRIVATE,Mercedes-Benz,E-Класс,COUPE_HARDTOP,...,0,0,0,1,0,1,1,0,0,0
343497,2000,380000,CONDITION_OK,4,OK,Москва,PRIVATE,Mercedes-Benz,M-Класс,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0
439784,2019,170000,CONDITION_OK,1,ERROR,Москва,PRIVATE,Skoda,Octavia,LIFTBACK,...,0,1,0,1,0,0,1,0,1,0
550845,2023,107,CONDITION_OK,1,OK,Краснодар,PRIVATE,Lada (ВАЗ),Largus,WAGON_5_DOORS,...,1,0,0,0,0,1,1,0,1,0
431808,2015,110727,CONDITION_OK,1,OK,Уфа,COMMERCIAL,Renault,Sandero,HATCHBACK_5_DOORS,...,1,1,0,0,0,1,1,0,0,0
167063,2024,0,CONDITION_OK,0,OK,Химки,COMMERCIAL,Genesis,GV80 Coupe,ALLROAD_5_DOORS,...,0,0,0,1,0,1,1,1,1,1
391523,2022,13000,CONDITION_OK,0,OK,Владивосток,COMMERCIAL,Nissan,X-Trail,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0
210052,2024,100,CONDITION_OK,1,OK,Новосибирск,PRIVATE,Hyundai,Elantra,SEDAN,...,0,0,0,0,0,0,0,0,0,0
46816,2024,34,CONDITION_OK,1,OK,Москва,COMMERCIAL,BMW,X5,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0


In [19]:
import pandas as pd
import numpy as np
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform

numeric_df = full_df.select_dtypes(include=[np.number])
numeric_df = numeric_df.loc[:, numeric_df.nunique() > 1]

corr = numeric_df.corr().abs().fillna(0)

dist = 1 - corr

dist = (dist + dist.T) / 2
np.fill_diagonal(dist.values, 0)

dist = dist.clip(lower=0)

dist_vect = squareform(dist)
Z = hierarchy.linkage(dist_vect, method='average')

max_d = 0.4
clusters = hierarchy.fcluster(Z, t=max_d, criterion='distance')

cluster_df = pd.DataFrame({'feature': numeric_df.columns, 'cluster': clusters})
to_keep = ['production_year', 'mileage', 'owners_number', 'doors_count', 'seats', 'engine_displacement', 'engine_power', 'fuel_rate']
for c in cluster_df['cluster'].unique():
    members = cluster_df.loc[cluster_df['cluster'] == c, 'feature']
    avg_corr = corr.loc[members, members].mean().sort_values(ascending=False)
    to_keep.append(avg_corr.index[0])
to_keep = list(dict.fromkeys(to_keep))

to_drop = [col for col in numeric_df.columns if col not in to_keep]
df_reduced = full_df.drop(columns=to_drop)

print(f"Удалено числовых коррелирующих признаков: {len(to_drop)}")
print(f"Осталось признаков в df_reduced: {df_reduced.shape[1]} из {full_df.shape[1]}")

Удалено числовых коррелирующих признаков: 112
Осталось признаков в df_reduced: 304 из 416


In [20]:
df_reduced.sample(10)

,production_year,mileage,condition,owners_number,accidents_resolution,region,seller_type,brand,model,body_type,...,sport-pedals,tja,isofix-front,21-inch-wheels,leather-gear-stick,steel-wheels,ptf,central-airbag,body-kit,projection-display
518746,2007,33000,CONDITION_OK,1,ERROR,Аксай,PRIVATE,Lada (ВАЗ),2107,SEDAN,...,0,0,0,0,0,0,0,0,0,0
224119,2021,23211,CONDITION_OK,1,OK,Москва,PRIVATE,Hyundai,Solaris,SEDAN,...,0,0,0,0,0,0,1,0,0,0
351020,2021,6122,CONDITION_OK,1,OK,Уссурийск,COMMERCIAL,Mini,Countryman,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0
424138,2016,4500,CONDITION_OK,1,ERROR,Погар,PRIVATE,Renault,Kaptur,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0
89776,2011,245600,CONDITION_OK,3,OK,Тверь,PRIVATE,Chevrolet,Aveo,SEDAN,...,0,0,0,0,0,0,0,0,0,0
297634,2023,18300,CONDITION_OK,1,OK,Москва,PRIVATE,Lexus,RX,ALLROAD_5_DOORS,...,0,0,0,1,1,0,1,0,0,0
449720,2024,0,CONDITION_OK,0,OK,Москва,COMMERCIAL,Soueast,S07,ALLROAD_5_DOORS,...,0,1,0,0,0,0,1,0,0,0
49847,2020,121000,CONDITION_OK,1,OK,Москва,PRIVATE,BMW,X6,ALLROAD_5_DOORS,...,0,0,0,1,1,0,1,0,1,1
48872,2024,0,CONDITION_OK,0,OK,Краснодар,COMMERCIAL,BMW,X6,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0
846,1999,363500,CONDITION_OK,3,OK,Уфа,PRIVATE,Alfa Romeo,166,SEDAN,...,0,0,0,0,0,0,1,0,0,0


### Добавим новые признаки

In [21]:
from sklearn.preprocessing import PolynomialFeatures

In [22]:
df_mod = df_reduced.copy()

In [23]:
df_mod["age"] = 2025 - df_mod["production_year"]

In [24]:
numeric_cols = ['mileage', 'engine_displacement', 'engine_power', 'age']

In [25]:
df_mod['log_price'] = np.log1p(df['price'])

In [26]:
df_mod.drop(columns=['price'], inplace=True)

Прологарифмируем некоторые признаки

In [27]:
for col in numeric_cols:
    df_mod[f'log_{col}'] = np.log1p(df_mod[col])

Возьмем корни от числовых признаков

In [28]:
for col in numeric_cols:
    df_mod[f'square_{col}'] = df_mod[col]**2

Полиномиальные преобразования

In [29]:
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_array = poly.fit_transform(df_mod[numeric_cols].fillna(0))
poly_feature_names = poly.get_feature_names_out(numeric_cols)
poly_df = pd.DataFrame(poly_array, columns=poly_feature_names, index=df_mod.index)

In [30]:
poly_new = poly_df.drop(columns=[c for c in poly_df.columns if c in numeric_cols])
df_mod = pd.concat([df_mod, poly_new.add_prefix('poly_')], axis=1)

Базовые отношения двигателя

In [31]:
df_mod['power_to_disp'] = df_mod['engine_power'] / df_mod['engine_displacement'].clip(lower=0.01)

Арифметичесике преобразования

In [32]:
# Произведения признаков
df_mod['mileage_x_power'] = df_mod['mileage'] * df_mod['engine_power']
df_mod['disp_x_power'] = df_mod['engine_displacement'] * df_mod['engine_power']
df_mod['owners_x_power'] = df_mod['owners_number'] * df_mod['engine_power']
df_mod["age_x_power"]    = df_mod["age"] * df_mod["engine_power"]
df_mod["age_x_mileage"]  = df_mod["age"] * df_mod["mileage"]

# Частные признаков
df_mod['power_div_mileage'] = df_mod['engine_power'] / df_mod['mileage'].clip(lower=0.01)
df_mod['disp_per_mile'] = df_mod['engine_displacement'] / df_mod['mileage'].clip(lower=0.01)
df_mod['mileage_per_hp'] = df_mod['mileage'] / df_mod['engine_power'].clip(lower=0.01)
df_mod['milage_per_year'] = df_mod['mileage'] / df_mod['age'].clip(lower=0.01)
df_mod['power_div_owners'] = df_mod['engine_power'] / df_mod['owners_number'].clip(lower=0.01)
df_mod['disp_div_owners'] = df_mod['engine_displacement'] / df_mod['owners_number'].clip(lower=0.01)
df_mod['disp_div_fuel'] = df_mod['engine_displacement'] / df_mod['fuel_rate'].clip(lower=0.01)
df_mod['fuel_div_disp'] = df_mod['fuel_rate'] / df_mod['engine_displacement'].clip(lower=0.01)
df_mod['power_div_fuel'] = df_mod['engine_power'] / df_mod['fuel_rate'].clip(lower=0.01)
df_mod['fuel_div_power'] = df_mod['fuel_rate'] / df_mod['engine_power'].clip(lower=0.01)

# Лог-взаимодействие
df_mod['log_mileage_x_log_power'] = df_mod['log_mileage'] * df_mod['log_engine_power']
df_mod['log_age_x_log_power'] = df_mod['log_age'] * df_mod['log_engine_power']

# Разности признаков
df_mod['power_minus_disp'] = df_mod['engine_power'] - df_mod['engine_displacement']

In [33]:
from pandas.api.types import is_numeric_dtype

# Отношение дверей к местам
df_mod['seats_num'] = df_mod['seats'].str.extract(r'(\d+)').astype(float)
df_mod['doors_to_seats_ratio'] = df_mod['doors_count'] / df_mod['seats_num'].clip(lower=0.01)

# Индикатор аварий
df_mod['has_accident'] = (
    ~df_mod['accidents_resolution'].fillna('none').str.lower().isin(['none', 'no', 'unknown'])
).astype(int)

binary_cols = [
    col for col in df.columns
    if is_numeric_dtype(df[col]) and
       set(df[col].dropna().unique()).issubset({0, 1})
]
df_mod['num_binary_features'] = df[binary_cols] \
    .astype(bool).sum(axis=1)

# Счетчики безопасности и комфорта
safety_keywords = ['airbag', 'detection', 'vsm', 'ldw', 'tja', 'rcta', 'ptf']
safety_cols = [c for c in df_mod.columns if any(k in c.lower() for k in safety_keywords)]
df_mod['safety_feature_count'] = df_mod[safety_cols] \
    .astype(bool).sum(axis=1)

comfort_keywords = ['heat', 'leather', 'massage', 'climate', 'vent', 'seat']
comfort_cols = [c for c in df_mod.columns if any(k in c.lower() for k in comfort_keywords)]
df_mod['comfort_feature_count'] = df_mod[comfort_cols] \
    .astype(bool).sum(axis=1)

# Количество preset-тегов
preset_cols = [c for c in df_mod.columns if c.startswith('preset')]
df_mod['num_presets'] = df_mod[preset_cols] \
    .astype(bool).sum(axis=1)

# Является ли машина новой
df_mod['is_new'] = ((df_mod['mileage'] < 1000) & (df_mod['owners_number'] <= 1)).astype(int)
df_mod['is_very_old'] = (df_mod['age'] > 20).astype(int)

df_mod['is_one_owner'] = (df_mod['owners_number'] == 1).astype(int)

Проверим, что нет дубликатов

In [34]:
df_mod.columns.duplicated().sum()

np.int64(0)

Проверим, что не появилось пустых значений

In [35]:
cols_with_na = df_mod.columns[df_mod.isna().any()].tolist()
print("Колонки с пропусками:", cols_with_na)

Колонки с пропусками: []


Сохраним датасет с новыми признаками

In [36]:
df_mod.to_csv('dataset_with_new_features.csv', index=False)

### Обучим LightGBM

In [37]:
!pip install lightgbm --install-option=--gpu --install-option="--opencl-include-dir=/usr/local/cuda/include/" --install-option="--opencl-library=/usr/local/cuda/lib64/libOpenCL.so"


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --install-option


In [38]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 19.4 MB/s eta 0:00:00


In [44]:
import optuna
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
import joblib

In [40]:
RANDOM_STATE = 42

In [45]:
df_shuffled = df_mod.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

n_total   = len(df_shuffled)
n_train   = int(0.2 * n_total)

train_df  = df_shuffled.iloc[:n_train] # 20%
test_df   = df_shuffled.iloc[n_train:] # 80%

print(train_df.shape, test_df.shape)

(120809, 352) (483237, 352)


In [46]:
X = train_df.drop('log_price', axis=1)
y = train_df['log_price']
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

cut_test_df = test_df.sample(frac=0.3, random_state=RANDOM_STATE)
X_test = cut_test_df.drop('log_price', axis=1)
y_test = cut_test_df['log_price']

num_features = X.select_dtypes(include=['int64','float64','int8']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_features),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features),
])

In [47]:
X_train_proc = preprocessor.fit_transform(X_train)
X_valid_proc = preprocessor.transform(X_valid)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [48]:
import lightgbm as lgb

N_TRIALS = 50

def objective(trial):
    params = {
        'device':           'gpu',
        'n_estimators':     trial.suggest_int('n_estimators', 100, 2000),
        'learning_rate':    trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'num_leaves':       trial.suggest_int('num_leaves', 31, 256),
        'max_depth':        trial.suggest_int('max_depth', 5, 30),
        'subsample':        trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'reg_alpha':        trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda':       trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50),
        'random_state':     RANDOM_STATE,
        'verbose':          -1,
    }
    model = LGBMRegressor(**params)
    model.fit(
        X_train_proc, y_train,
        eval_set=[(X_valid_proc, y_valid)],
        callbacks=[lgb.early_stopping(stopping_rounds=N_TRIALS)]
    )
    preds = model.predict(X_valid_proc)
    return r2_score(y_valid, preds)

In [ ]:
study = optuna.create_study(direction='maximize', study_name='lgbm_opt_r2')
study.optimize(
    objective,
    n_trials=N_TRIALS,
    show_progress_bar=True,
)

best = study.best_params
best.update({'device':'gpu','random_seed':RANDOM_STATE,'verbose':-1})
final_model = LGBMRegressor(**best)
final_model.fit(X_train_proc, y_train, eval_set=(X_valid_proc, y_valid), early_stopping_rounds=N_TRIALS)

joblib.dump(final_model, 'drive/MyDrive/ML_project/best_lightgbm_model_with_new_feature.pkl')
joblib.dump(preprocessor, 'drive/MyDrive/ML_project/lightgbm_preprocessor_with_new_feature.pkl')
print("Models saved to best_lightgbm_model_with_new_feature.pkl and lightgbm_preprocessor_with_new_feature.pkl")

print("Best R²:", study.best_value)
print("Best params:", study.best_params)

[I 2025-06-10 14:28:42,277] A new study created in memory with name: lgbm_opt_r2


  0%|          | 0/50 [00:00<?, ?it/s]

<ipython-input-48-54d882b388f3>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate':    trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
<ipython-input-48-54d882b388f3>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample':        trial.suggest_uniform('subsample', 0.5, 1.0),
<ipython-input-48-54d882b388f3>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
<ipython-input-48-54d882b388f3>:14: FutureWarning: suggest_loguniform has been deprecated in v

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1250]	valid_0's l2: 0.0439678


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1991]	valid_0's l2: 0.0368983


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[I 2025-06-10 15:56:59,826] Trial 44 finished with value: 0.9688145064575516 and parameters: {'n_estimators': 1991, 'learning_rate': 0.0036149738682994077, 'num_leaves': 193, 'max_depth': 29, 'subsample': 0.7092202045338285, 'colsample_bytree': 0.6796398052218585, 'reg_alpha': 0.0014025156248740935, 'reg_lambda': 1.1628519141906803e-07, 'min_child_weight': 42}. Best is trial 25 with value: 0.975714953550876.


<ipython-input-48-54d882b388f3>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate':    trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
<ipython-input-48-54d882b388f3>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample':        trial.suggest_uniform('subsample', 0.5, 1.0),
<ipython-input-48-54d882b388f3>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
<ipython-input-48-54d882b388f3>:14: FutureWarning: suggest_loguniform has been deprecated in v

Training until validation scores don't improve for 50 rounds


In [ ]:
X_test_proc = preprocessor.transform(X_test)
y_pred = final_model.predict(X_test_proc)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print(f"Test MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"Test MSE: {mean_squared_error(y_test, y_pred):.4f}")
print(f"Test R^2: {r2_score(y_test, y_pred):.4f}")

### Выводы

$R^2 = 0.9757$

In [ ]:
param = {
    'n_estimators': 1165, 
    'learning_rate': 0.06555964500796287, 
    'num_leaves': 162, 
    'max_depth': 18, 
    'subsample': 0.8642338819071332, 
    'colsample_bytree': 0.5520540488251534, 
    'reg_alpha': 0.00015346592557518072, 
    'reg_lambda': 9.94216540977995e-05, 
    'min_child_weight': 16
}